In [1]:
import numpy as np
import torch as th
import os
%load_ext autoreload
%autoreload 2
from PIL import Image 
from IPython.display import clear_output
from matplotlib.pyplot import imshow 
import matplotlib.pyplot as plt
import PIL
import torch

from itertools import islice
from torchvision.transforms.functional import to_tensor

import pandas as pd
%matplotlib inline

# Add path to project 

In [2]:
import sys
sys.path
sys.path.append('/code/Deep-Vectorization-of-Technical-Drawings/')

In [3]:
from util_files.rendering.cairo  import render,render_with_skeleton
from util_files.data.graphics_primitives import PT_LINE,  PT_CBEZIER, PT_QBEZIER
import  util_files.loss_functions.supervised  as supervised_loss 
from util_files.optimization.optimizer.scheduled_optimizer import ScheduledOptimizer
import util_files.dataloading as dataloading
from vectorization import load_model

/code/Deep-Vectorization-of-Technical-Drawings/util_files/loss_functions/lovacz_losses.py:191: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if (classes is 'present' and fg.sum() == 0):


# Parameters

In [4]:
device = torch.device('cuda:{}'.format(0))
prefetch_data = True
batches_completed_in_epoch=0
epoch_size = 20000
curve_count = 10
model_type = "model"

# Data loading

In [5]:
def prepare_batch_on_device(batch_data, device):
    images, y_true = batch_data
    images = images.to(device)
    y_true = y_true.to(device)
    return images, y_true

In [6]:
make_loaders_fn = dataloading.prepare_loaders["abc"]

#data root is folder where folder abc with memory maped files are
loader_params = dict(data_root='/data/preprocessed/', train_batch_size=64,
                         val_batch_size=64,
                         mini_val_batches_n_per_subset=16,
                         memory_constraint=8e+9,
                         shuffle_train=True, prefetch=prefetch_data, device=device)


In [7]:
train_loader, val_loader, val_mini_loader = make_loaders_fn(**loader_params)

In [13]:
for batch_i, batch_data in islice(enumerate(train_loader), batches_completed_in_epoch, len(train_loader)):
    images, target =  prepare_batch_on_device(batch_data, device)
    break

Process Process-4:
Traceback (most recent call last):
  File "/home/vage/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/vage/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/vage/PycharmProjects/Deep-Vectorization-of-Technical-Drawings/util_files/data/chunked.py", line 76, in fill_next_chunk
    self.data.write_next_to(self.input_buffers[self.next_buffer_i], self.target_buffers[self.next_buffer_i])
  File "/home/vage/PycharmProjects/Deep-Vectorization-of-Technical-Drawings/util_files/data/chunked.py", line 96, in write_next_to
    inputs[:size] = self.dataset.images[self.head : self.head + size]
RuntimeError: The expanded size of the tensor (46960) must match the existing size (0) at non-singleton dimension 0.  Target sizes: [46960, 3, 64, 64].  Tensor sizes: [0, 3, 64, 64]


# Model loading 

In [ ]:
# in json file you can chose model type and it's parameters as in examples in vectorization/models/specs/ folder

In [10]:
# chose json file 
model_json_param = '/code/Deep-Vectorization-of-Technical-Drawings/vectorization/models/specs/resnet18_blocks1_bn_64__c2h__trans_heads1_feat256_blocks4_ffmaps512__h2o__out512.json'
#load/create model 
model = load_model(model_json_param).to(device)

In [11]:
model 

GenericVectorizationNet(
  (features): ResnetConv(
    (conv): Sequential(
      (0): Sequential(
        (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (1): MaybeModule(True) BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): LeakyReLU(negative_slope=0.01, inplace=True)
        (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      )
      (1): Sequential(
        (0): BasicBlock(
          (relu): LeakyReLU(negative_slope=0.01, inplace=True)
          (conv): Sequential(
            (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (1): MaybeModule(True) BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): LeakyReLU(negative_slope=0.01, inplace=True)
            (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (4): MaybeModule(True) Batc

# Loading Loss function

In [7]:
# Here you are chosing between 'vectran_loss','vectran_mapping_L2','vectran_mapping_L1','pseudoLap1_loop' ,
#'pseudoLap1_no_loop','lovasz_hinge'. Code for this losses you can find util_files.loss_functions.supervised.
make_loss_fn = supervised_loss.prepare_losses["vectran_loss"]

## Use case 

loss = make_loss_fn(y_pred, y_true, l2_weight=0.5)

# Optimizer

In [12]:
optimizer = ScheduledOptimizer(
    torch.optim.Adam(
        filter(lambda x: x.requires_grad, model.parameters()),
        betas=(0.9, 0.98), eps=1e-09),
    1000, 4000)

# Model training

In [ ]:
for epoch_i in range(epoch_size):
    for batch_i, batch_data in islice(enumerate(train_loader), batches_completed_in_epoch, len(train_loader)):
        images, target =  prepare_batch_on_device(batch_data, device) 
        out = model(images[:,:1,...],10)
        loss = make_loss_fn(y_pred, y_true, l2_weight=0.5)
        loss.backward()
        optimizer.step_and_update_lr()
        optimizer.zero_grad()